In [27]:
#get daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import os
df=pd.DataFrame()
for f in os.listdir(r'./DataSource/baiduMap'):
    print(f)
    d=pd.read_csv(r'./DataSource/baiduMap/'+f, on_bad_lines='skip')
    df=pd.concat([df,d])

# df=df.rename(columns={'抓取日期':'test','抓取时段':'抓取日期','路况语义化描述':'抓取时段'})
df['抓取日期']=pd.to_datetime(df['抓取日期']).dt.date
df=df.drop_duplicates(subset=['城市','商场名称','抓取日期','抓取时段'])

df.tail(2)

baiduTrafficV2_2022-12-01.csv
baiduTrafficV2_2022-12-02.csv
baiduTrafficV2_2022-12-03.csv
baiduTrafficV2_2022-12-04.csv
baiduTrafficV2_2022-12-05.csv
baiduTrafficV2_2022-12-06.csv
baiduTrafficV2_2022-12-07.csv
baiduTrafficV2_2022-12-08.csv
baiduTrafficV2_2022-12-09.csv
baiduTrafficV2_2022-12-10.csv
baiduTrafficV2_2022-12-11.csv
baiduTrafficV2_2022-12-12.csv
baiduTrafficV2_2022-12-13.csv
baiduTrafficV2_2022-12-14.csv
baiduTrafficV2_2022-12-15.csv
baiduTrafficV2_2022-12-16.csv
baiduTrafficV2_2022-12-17.csv
baiduTrafficV2_2022-12-18.csv
baiduTrafficV2_2022-12-19.csv
baiduTrafficV2_2022-12-20.csv
baiduTrafficV2_2022-12-21.csv
baiduTrafficV2_2022-12-22.csv
baiduTrafficV2_2022-12-23.csv
baiduTrafficV2_2022-12-24.csv
baiduTrafficV2_2022-12-25.csv
baiduTrafficV2_2022-12-26.csv
baiduTrafficV2_2022-12-27.csv
baiduTrafficV2_2022-12-28.csv
baiduTrafficV2_2022-12-29.csv
baiduTrafficV2_2022-12-30.csv
baiduTrafficV2_2022-12-31.csv
baiduTrafficV2_2023-01-01.csv
baiduTrafficV2_2023-01-02.csv
baiduTraff

,编号,省份,城市,城市线,商场名称,经度,纬度,抓取日期,抓取时段,路况语义化描述,路况状态,路径整体评价,附近道路数量,样本道路数量,平均通行速度（KM/H),radius
871,144,安徽,阜阳,三线及以下,万达广场(阜阳颍泉店),115.823352,32.939661,2023-01-04,10,该区域整体畅通。颍滨路：南向北，颍滨路附近行驶缓慢。,1.0,畅通,1,1,22.94,200
872,157,甘肃,陇南,三线及以下,兰天•莲湖广场,104.933504,33.396484,2023-01-04,10,NaN,NaN,NaN,0,0,NaN,200


In [31]:
old=pd.read_hdf(r'Processed_Data/mall.h5',key='0105')
new=pd.concat([old,df])
new=new.drop_duplicates()
new.to_hdf(r'Processed_Data/mall.h5',key='0105')

In [25]:
d1=pd.read_hdf('Processed_Data/Mallcity.h5','raw')
d2=pd.pivot_table(df[df['抓取日期'].astype('str')>'2023-01-01'],index=['城市线','城市','商场名称'],columns=['抓取日期','抓取时段'],values='路况状态',aggfunc='max').reset_index().fillna(1)
d3=d1.merge(d2,how='left',on=['城市线', '城市', '商场名称'])
d3.to_clipboard(index=False)

In [83]:
d=df.copy()
c1=d['城市'].isin(d1['城市'].drop_duplicates())
c2=d['省份'].isin(d1['城市'].drop_duplicates())

d=d[c1|c2]

c3=d['抓取时段'].isin([10,12,14])
d=d[~c3]

d_N=d.groupby(by=['抓取日期','抓取时段'])['编号'].count().reset_index().rename(columns={'编号':'N'})

d=d.groupby(by=['抓取日期','抓取时段','路径整体评价'])['编号'].count().reset_index()

d=d[d['路径整体评价']=='畅通']
d=d.merge(d_N,on=['抓取日期','抓取时段'],how='left')
d['拥堵率']=1-(d['编号']/d['N'])

d=d.sort_values(by=['抓取日期','抓取时段'],ascending=True)
d['拥堵率_last7d_mean']=d.groupby(by='抓取时段')['拥堵率'].transform(lambda x: x.rolling(7, 1).mean())
pd.pivot_table(d,index=['抓取日期'],columns='抓取时段',values='拥堵率_last7d_mean',aggfunc='mean').to_clipboard(header=False)

抓取时段,16,20
抓取日期,,
2022-12-01,NaN,0.013514
2022-12-02,0.027027,0.030405
2022-12-03,0.037162,0.013514
2022-12-04,0.057432,0.013514
2022-12-05,0.020270,0.010135
2022-12-06,0.033784,0.027027
2022-12-08,0.016835,0.020202
2022-12-09,0.043771,0.043771
2022-12-10,0.050505,0.040404
